# Imports

In [ ]:
# Uncomment the line below and execute this cell if you are missing the modules required in this notebook
#%pip install requests httpx numpy polars scipy matplotlib

In [ ]:
import base64
import io
import os
import gzip
import json
import requests
from requests.auth import HTTPBasicAuth

import numpy as np
import polars as pl
from scipy.special import log_softmax
import matplotlib.pyplot as plt
import httpx

# Fetch the reference promoter from NCBI

We’ll compute a promoter window around the LDLR TSS. Using the ClinGen gene page (GRCh38 LDLR coordinates), we’ll set TSS = 11089463 (forward strand) and take +- 5kbp range. Then we’ll fetch from NC_000019.10 (GRCh38 chr19) using E-utilities efetch

In [ ]:
NCBI_NUCCORE_ACC = "NC_000019.10"      # GRCh38 chr19
LDLR_TSS = 11089463                    # https://search.clinicalgenome.org/kb/genes/HGNC:6547/by-disease
RANGE = 5_000
PROMOTER_START = LDLR_TSS - RANGE
PROMOTER_END   = LDLR_TSS + RANGE - 1
STRAND = 1                             # forward

In [ ]:
def fetch_fasta_region_ncbi(acc: str, start: int, end: int, strand: int=1) -> str:
    """Fetch FASTA for a genomic slice via NCBI efetch"""
    
    url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    params = dict(db="nuccore", id=acc, rettype="fasta", retmode="text",
                  seq_start=start, seq_stop=end, strand=strand)
    with httpx.Client(timeout=60) as cx:
        r = cx.get(url, params=params)
        r.raise_for_status()
    return r.text

def parse_fasta_sequence(fasta_text: str) -> str:
    lines = [ln.strip() for ln in fasta_text.splitlines() if ln.strip()]
    seq_lines = [ln for ln in lines if not ln.startswith(">")]
    return "".join(seq_lines).upper()

In [ ]:
fasta_txt = fetch_fasta_region_ncbi(NCBI_NUCCORE_ACC, PROMOTER_START, PROMOTER_END, STRAND)
ref_seq = parse_fasta_sequence(fasta_txt)
print(f"Fetched promoter window: {NCBI_NUCCORE_ACC}:{PROMOTER_START}-{PROMOTER_END} len={len(ref_seq)} bp")

# Call Evo 2 NIM /forward and decode logits

The Evo 2 NIM endpoints specify `/biology/arc/evo2/forward` with `output_layers=["output_layer"]`. The response returns a Base64-encoded NPZ; the output_layer array has shape `[seq_len, batch, 512]`. Map logits indices to bases using the ASCII mapping given in the docs: `A=65, C=67, T=84, G=71`.

In [ ]:
host = '<host>' # TODO: Fill in with the deployed NIM URL, without http://, https:// and forward slashes
username = '<username>' # TODO: Fill in with username from
password = '<password>' # TODO: Fill in with password from deployment

EVO2_URL = f'https://{username}:{password}@{host}'
FORWARD_EP=f"{EVO2_URL}/biology/arc/evo2/forward"


In [ ]:
# ASCII indices for DNA in Evo 2 tokenizer
IDX = {"A": 65, "C": 67, "T": 84, "G": 71}
DNA = set("ACTG")

In [ ]:
def evo2_forward_logits(sequence: str) -> np.ndarray:
    """
    Call Evo2 NIM forward endpoint and return [L, 4] log-probabilities for A,C,T,G at each position,
    computed from output_layer logits via log_softmax.
    """
    assert set(sequence).issubset(DNA), "Sequence must be A/C/T/G only"

    payload = {"sequence": sequence, 'output_layers': ['output_layer']}
    
    r = requests.post(FORWARD_EP, json=payload, timeout=120)
    r.raise_for_status()
    data = r.json()  # { "data": "<base64 npz>", "elapsed_ms": ... }

    arr = np.load(io.BytesIO(base64.b64decode(data["data"])))
    logits = arr["output_layer.output"]  # [L, B, 512]

    if logits.ndim != 3 or logits.shape[1] != 1:
        raise ValueError(f"Unexpected logits shape {logits.shape}")

    logits = logits[:, 0, :]  # [L, 512]

    logp512 = log_softmax(logits, axis=1)
    logp4 = np.vstack([
        logp512[:, IDX["A"]],
        logp512[:, IDX["C"]],
        logp512[:, IDX["T"]],
        logp512[:, IDX["G"]],
    ]).T

    return logp4


In [ ]:
logp4 = evo2_forward_logits(ref_seq)

# In-silico saturation mutagenesis (ΔlogP per SNV)

For each position, compute ΔlogP = logP(ALT) − logP(REF). Negative values suggest evo-prior intolerance at that position/base swap.

In [ ]:
BASES = ("A", "C", "T", "G")
bidx = {b: i for i, b in enumerate(BASES)}
ref_arr = list(ref_seq)

In [ ]:
# ΔlogP table: for each pos, each possible ALT != REF
records: list[tuple[int, int, str, str, float]] = []
for i, refb in enumerate(ref_arr):
    for altb in BASES:
        if altb == refb:
            continue
        dlogp = float(logp4[i, bidx[altb]] - logp4[i, bidx[refb]])
        records.append((i, int(PROMOTER_START + i), refb, altb, dlogp))

df = pl.DataFrame(
    records,
    schema={
        "pos0": pl.Int64,
        "genomic_pos": pl.Int64,
        "ref": pl.Utf8,
        "alt": pl.Utf8,
        "delta_logp": pl.Float64,
    },
    orient="row",
).with_columns(
    (pl.col("pos0") - RANGE).alias("pos_rel_TSS"),
    pl.lit("LDLR").alias("gene"),
    pl.lit("GRCh38").alias("assembly"),
    pl.lit("NC_000019.10").alias("chrom"),
)

In [ ]:
df

In [ ]:
print(df.sort("delta_logp").head(10))

# Overlay ClinVar variants in this window

ClinVar provides TSV summaries and VCFs. For a simple overlay, we can pull variant_summary.txt.gz and filter to LDLR + our coordinate window

In [ ]:
!wget https://ftp.ncbi.nlm.nih.gov/pub/clinvar/tab_delimited/variant_summary.txt.gz

In [ ]:
!gunzip variant_summary.txt.gz

In [ ]:
cv = pl.read_csv('variant_summary.txt', separator='\t', infer_schema_length=100_000, null_values=['na'])

In [ ]:
# 1) Narrow ClinVar to your window + GRCh38 + LDLR, and expose VCF fields
cv_prom = (
    cv.filter(
        (pl.col("GeneSymbol") == "LDLR") &
        (pl.col("Assembly") == "GRCh38") &
        (pl.col("Chromosome").cast(str) == "19") &
        (pl.col("Start") >= PROMOTER_START) &
        (pl.col("Start") <= PROMOTER_END)
    )
    .with_columns([
        pl.col("Type").str.to_lowercase().alias("type_lc"),
        pl.col("PositionVCF").cast(pl.Int64).alias("pos_vcf"),
        pl.col("ReferenceAlleleVCF").alias("ref_vcf"),
        pl.col("AlternateAlleleVCF").alias("alt_vcf"),
        pl.col("ChromosomeAccession").alias("chrom_acc"),
    ])
)

In [ ]:
# 2) Only SNVs for base-by-base overlay (indels use left-justified pos; different handling)
cv_snvs = cv_prom.filter(pl.col("type_lc") == "single nucleotide variant")

In [ ]:
# 3) Join Evo2 ΔlogP (df has columns: genomic_pos, ref, alt) to ClinVar using VCF fields
#    For SNVs, HGVS vs VCF positions coincide, so this is safe.
df_join = (
    df.join(
        cv_snvs.select([
            "pos_vcf","ref_vcf","alt_vcf","ClinicalSignificance","ReviewStatus",
            "VariationID","Name","Type","RS# (dbSNP)"
        ]),
        left_on=["genomic_pos","ref","alt"],
        right_on=["pos_vcf","ref_vcf","alt_vcf"],
        how="left"
    )
)

In [ ]:
df_join.filter(pl.col("ClinicalSignificance").is_not_null()).sort("genomic_pos")

# Analysis & visualization

In [ ]:
# Worst-case ΔlogP at each position (most “disruptive” ALT)
per_pos_min = (
    df.group_by("pos0")
      .agg(pl.col("delta_logp").min().alias("min_delta_logp"))
      .sort("pos0")
)
x = per_pos_min["pos0"].to_numpy() - RANGE
y = per_pos_min["min_delta_logp"].to_numpy()

In [ ]:
plt.figure(figsize=(10,3))
plt.axvline(0, linestyle="--")  # TSS
plt.plot(x, y)
plt.xlabel("Position relative to TSS (bp)")
plt.ylabel("min ΔlogP across SNVs")
plt.title("Evo2 zero-shot prior: intolerance to SNVs in LDLR promoter")
plt.tight_layout()
plt.show()